# 1 - Présentation
## 1.1 - Introduction
Les réseaux sociaux sont une source de données presques infinis. Avec les Tweets sur Twitter ou les murs sur Facebook, il y a une multitude d'informations en attente d'être étudiées.


Le projet est une application qui permet de lire les tweets sur twitter. Durant cette lecture, le programme va définir si un message est positif ou négatif. Ce qui va permettre de définir une réputation.

L'objectif de ce projet est de créer un programme qui comprends si un tweet est positif ou négatif. Cela peut servir pour définir la réputation d'une entreprise sur les réseaux sociaux.

Le but de ce jeu de données est de prédire si un tweet est positif, neutre ou négatif. L'exercice nécessite de réaliser une analyse de sentiment en réalisant une classification à 3 sorties.

## 1.2 - Source de Données
La source de données est disponible via le lien suivant : http://help.sentiment140.com/for-students/

Les données sont des tweets avec divers métadata pour compléter l'information.

# 2 - Préparation des données
## 2.1 - Charger les librairies
Ci-dessous la liste des librairies nécessaires durant l'analyse des données :

In [ ]:
####################
# Définir la liste des librairies nécessaire pour les différentes
# étapes du traitement
####################

import pandas as pd
import nltk
import unicodedata
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [ ]:
####################
# Chargement du fichier d'apprentissage d'entraintement
####################

dataSource = pd.read_csv('./data/training.csv', 
                 header=0,
                 sep=',',
                 names=['Sentimental','Id','Date','Query', 'Owner','Tweet'],
#                 nrows = 5000000,
                 encoding = 'latin-1'
)

Maintenant nous allons afficher les 20 premières lignes de ce dataset pour mieux appréhender l'information :

In [ ]:
####################
# Découverte des données
####################

# Charger 10000 tweets négatifs
dataNegative = dataSource[-5000:]

# Charger 10000 tweets positifs
dataPositive = dataSource[:5000]
 
# Concatener les deux éléments à fin d'en créer un seul
df = pd.concat([dataPositive, dataNegative])

# Afficher les 20 premières lignes
df.head(20)

Il  y a 6 dimensions dans ce dataset :
* Le sentiment du tweet (0 = negatif, 2 = neutre et 4 = positif) ;
* L'id du tweet ;
* La date de publication ;
* La requête. (S'il n'y en a aucune alors NO_QUERY) ;
* L'utilisateur qui a tweeté ;
* Le texte du tweet.

Nous allons afficher les types des données chargées :

In [ ]:
df.dtypes

Les types des données sont corrects pour l'analyse. Il n'est pas nécessaire de les casts.

## 2.3 - Nettoyer les données
Dans l'objectif d'analyser les données, il faut tout d'abord les nettoyer afin qu'elles soient le plus facile à lire pour l'algorithme.

Durant cette partie nous allons définir des règles pour le nettoyage de données non structurée au format texte.

In [ ]:
####################
# Nettoyer les données
####################

# Créer une fonction qui va appliquer toutes les règles de nettoyage
def dataClean(column):
    # Définir la ponctuation à supprimer
    signs = ['.', ',' ,'!', '?', ';', "'", '-', '_', '&', '(', ')', '*']
    
    # Transformer les caractères en minuscule
    column = column.str.lower()
    # Supprimer les utilisateurs
    column = column.str.replace('@[\w]*', '')
    # Supprimer les tags
    column = column.str.replace('#[\w]*', '')
    
    # Supprimer les signes
    for sign in signs:
        column = column.str.replace(sign, '')
    
    # Supprimer les liens hypertextes
    column = column.str.replace('/^(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)*\/?$/', '')
    # Supprimer les espaces en début et fin de mot
    column = column.str.strip()
    
    return column

# Appliquer la fonction de nettoyage aux données
df['Tweet'] = dataClean(df['Tweet'])

Cette fonction va permettre de préparer les données pour la suite de l'analyse.

Les actions réalisées :
* Mettre le texte en minuscule afin d'éviter de compter certains mots en double comme par exemple : Hasard et hasard. C'est le même mot pour nous en le lisant mais pour l'algorithme cela est différent ;
* Supprimer les pseudos Twitter ;
* Supprimer les caractères comme le "." ou le "!" ;
* Supprimer les URL ;
* Supprimer les espaces avant et après le paragraphe.

## 2.4 - Preprocessing
Maintenant que nous allons nettoyer les données, il faut adapter l'information afin de la rendre exploitable par les algorithmes de Machine Learning.

<span style="color:orange">/!\ Lancer ces commandes si c'est la première fois que vous utilisez NLTK :
* <span style="color:orange">nltk.download('punkt')</span>
* <span style="color:orange">nltk.download("stopwords")</span>
* <span style="color:orange">nltk.download('wordnet')</span>

In [ ]:
####################
# Transformer les données
####################

# Définir la fonction de préprocessing
def preprocessing(column):
    wordnet_lemmatizer = WordNetLemmatizer()
    # Tokenizer les tweets
    column = column.apply(lambda row: nltk.word_tokenize(row))
    # Supprimer les stopwords
    column = column.apply(lambda x: [item for item in x if item not in stopwords.words('english')])
    # Appliquer le lemmatizer
    column = column.apply(lambda x: [wordnet_lemmatizer.lemmatize(item) for item in x])

    return column

# Appliquer la fonction sur les données
df['TweetTK'] = preprocessing(df['Tweet'])    

Cette fonction va permettre de **Tokenize** les tweets, faire disparaitre les **Stop Words** puis d'appliquer un algorithme **Lemmatisation**.

## 2.5 - Bag of Words

In [ ]:
####################
# Transformer les données
####################

# Définir une fonction qui permet de créer un tableau
# qui contient chaque mot de chaque tweet
def getListofWords(tweets):
    allWords = []
    for words in tweets:
        allWords.extend(words)
    return allWords

# Définir une fonction qui supprimer tout les doublons d'un tableau
def getWordFeatures(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def findFeatures(document, wordFeatures):
    features = {}
    for w in wordFeatures:
        features[w] = (w in document)
    return features

listWords = getListofWords(df['TweetTK'])
wordFeatures = getWordFeatures(listWords)
df['TweetFeatures'] = df['TweetTK'].apply(lambda x: findFeatures(x, wordFeatures))

## 2.6 - Conclusion
Les données sont maintenant exploitables pour commencer l'étape d'apprentissage en Machine Learning.

# 3. Apprentissage
Dans cette partie nous allons apprendre à l'algorithme NaiveBayes à reconnaitre si un tweet est positif ou négatif.

In [ ]:
####################
# Phrase d'apprentissage de l'algorithme 
####################

trainingSet = []

for words, sentimental in zip(df['TweetFeatures'], df['Sentimental']):
     for k, v in zip(words, words.values()):
        trainingSet.append(({k: v}, sentimental))

classifier = nltk.NaiveBayesClassifier.train(trainingSet)

# 4. Vérification
Dans cette partie, nous allons vérifier la qualité de l'apprentissage sur un jeu de données de test.

In [ ]:
####################
# Phrase de test de l'algorithme 
####################

datasetTest = pd.read_csv('./data/test.csv', 
                 header=0,
                 sep=',',
                 names=['Sentimental','Id','Date','Query', 'Owner','Tweet'],
                 encoding = 'latin-1'
)

datasetTest['Tweet'] = dataClean(datasetTest['Tweet'])
datasetTest['TweetTK'] = preprocessing(datasetTest['Tweet'])

listWords = getListofWords(datasetTest['TweetTK'])
wordFeatures = getWordFeatures(listWords)
datasetTest['TweetFeatures'] = datasetTest['TweetTK'].apply(lambda x: findFeatures(x, wordFeatures))

testSet = []

for words, sentimental in zip(datasetTest['TweetFeatures'], datasetTest['Sentimental']):
    for k, v in zip(words, words.values()):
        testSet.append(({k: v}, sentimental))

Nous allons maintenant vérifier le résultat sur un dataset de test pour connaitre le % de bonne réponses.

La fonction ci-dessous permet de tester la véracité de l'apprentissage avec les données de tests.

In [ ]:
# Permettre de vérifier l'apprentissage avec les données de tests tout en mesurant
# les performances via un pourcentage.
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testSet))*100)

Le taux de bonnes réponses est d'environ 36% avec 20000 tweets.

Malheureusement l'ordinateur qui fait tourner ce script ne peut prendre en charge les 1 600 000 tweets de la base de tests avec cet algorithme.

# 5. Conclusion

L'algorithme de ML Naive Bayes fonctionne avec un taux de 36% en moyenne de bonnes réponses sur le dataset de test. Malheureusement je ne peux charger que 20000 lignes du dataset d'entrainement. Mon ordinateur ne tient pas la charge. Je ne peux donc pas vérifier si je peux atteindre un taux de réussite plus important avec cette méthodologie.


# 6. Continuation
La continuation du projet serait de trouver pourquoi nous avons un taux de réussite aussi faible et d'améliorer le modèle.

Une autre piste est d'améliorer le code afin d'essayer de mieux gérer la ram afin de pouvoir prendre plus de lignes dans le traininset.


# 7. Notation Personnelle

Je me donnerai une note entre 13 et 14 pour le rendu effectué.

Cette notation comprends notamment le nettoyage des données, le preprocessing, le bag of words ainsi que l'utilisation de l'algorithme de machine learning Nayves Baye.

Je n'ai malheureusent pas réussir à obtenir un résultat satisfaisant pour le machine learning avec un taux de 36% de bonnes réponses sur le dataset de test. Ce chiffre restre très faible et n'est malheureusement pas satisfaisant pour une application dans un environnement de production.

